In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
from PIL import Image
import os
from pytorch_tcn import TCN
import time

**DATASET Definition**

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.real_dir = real_dir
        self.fake_dir = fake_dir
        self.real_images = [f for f in os.listdir(real_dir) if not os.path.isdir(os.path.join(real_dir, f))]
        self.fake_images = [f for f in os.listdir(fake_dir) if not os.path.isdir(os.path.join(fake_dir, f))]
        self.transform = transform

    def __len__(self):
        return len(self.real_images) + len(self.fake_images)

    def __getitem__(self, idx):

      if idx < len(self.real_images):
        img_path = os.path.join(self.real_dir, self.real_images[idx])
        label = 0  # Real
      else:

        img_path = os.path.join(self.fake_dir, self.fake_images[idx - len(self.real_images)])
        label = 1  # Fake

      #Debugging print
      print(f"Processing file: {img_path}")

      # Checking if img_path is a directory
      if os.path.isdir(img_path):
        raise ValueError(f"Expected a file but found a directory: {img_path}")

      image = Image.open(img_path).convert('L')  # Convert to grayscale

      if self.transform:
        image = self.transform(image)

      return image, label

**DATA Loader**

In [ ]:
from torchvision import transforms
import os

real_dir = '/data/mpstme-rutuja/Output/Melspec/real_melspec_13k'
fake_dir = '/data/mpstme-rutuja/Output/Melspec/fake_melspec_13k'

# Image transformations (resizing and converting to tensor)
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()          
])

# Creating labelled dataset 
dataset = SpectrogramDataset(real_dir, fake_dir, transform=transform)

# Spliting into training-testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)